# Extração dos dados do Enem de 2019

**1 - Importa as bibliotecas necessárias para rodar esse notebook.**

In [7]:
import pandas as pd
import numpy as np
import zipfile
import gc
import requests
from io import BytesIO
import os

**2 - Define o nome que vamos utilizar para a pasta de armazenamento dos arquivos.**

In [2]:
folder_enem = 'enem2019'

**3 - Cria o diretório, caso ele não exista.**

In [3]:
# Cria o diretório que vai armazenar os artefatos que serão baixados do Enem 2019, caso ele não exista.
os.makedirs(f'./{folder_enem}', exist_ok=True)

**4 - Define a URL onde os dados serão baixados.**

In [4]:
url = "http://download.inep.gov.br/microdados/microdados_enem_2019.zip"

**5 - Faz o download do arquivo compactado.**

In [5]:
%%time
filebytes = BytesIO(
    requests.get(url, stream=True).content
)

Wall time: 4min 43s


**6 - Descompacta o arquivo e extrai o conteúdo dentro da pasta que definimos para esse notebook.**

In [6]:
%%time
myzip = zipfile.ZipFile(filebytes)
myzip.extractall(f'./{folder_enem}')

del(filebytes)

Wall time: 34.9 s


**7 - Insere os dados do Enem 2019 no DataFrame.**

In [8]:
%%time
df = pd.read_csv(
    f"./{folder_enem}/DADOS/MICRODADOS_ENEM_2019.csv",
    sep=";",
    decimal=",", 
    encoding='latin-1'
)

Wall time: 2min 46s


In [9]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001004627,2019,1506807,Santarém,15,PA,21,M,1,3,...,A,A,A,C,B,A,D,A,B,A
1,190001004628,2019,1504059,Mãe do Rio,15,PA,16,F,1,3,...,A,A,A,B,B,A,B,A,A,A
2,190001004629,2019,1505502,Paragominas,15,PA,18,F,1,1,...,B,A,A,D,B,B,D,A,C,B
3,190001004630,2019,1507706,São Sebastião da Boa Vista,15,PA,23,M,0,3,...,A,A,A,C,A,A,D,A,A,A
4,190001004631,2019,1503903,Juruti,15,PA,23,M,1,3,...,A,A,A,B,A,A,D,A,A,A


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5095270 entries, 0 to 5095269
Columns: 136 entries, NU_INSCRICAO to Q025
dtypes: float64(20), int64(71), object(45)
memory usage: 5.2+ GB


In [11]:
# Limpa a memória
print("gc: ", gc.collect())

gc:  7


**8 - Cria um novo DataFrame com os dados do DF original filtrados pelos alunos que residem em MG.**

In [12]:
%%time
df_mg = df.query('SG_UF_RESIDENCIA == "MG"')

Wall time: 757 ms


In [13]:
len(df_mg)

538896

In [14]:
df_mg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 538896 entries, 1044 to 5095260
Columns: 136 entries, NU_INSCRICAO to Q025
dtypes: float64(20), int64(71), object(45)
memory usage: 563.3+ MB


**9 - Salva o conteúdo do DataFrame filtrado em um arquivo CSV.**

In [33]:
%%time
df_mg.to_csv( f"./{folder_enem}/MICRODADOS_ENEM_2019_MG.csv", sep=';', encoding='utf-8', index=False)

Wall time: 53.9 s


**10 - Exclui o arquivo original do Enem 2019 com mais de 3gb.**

In [34]:
%%time
os.remove(f"./{folder_enem}/DADOS/MICRODADOS_ENEM_2019.csv")

Wall time: 146 ms


**Final da etapa de extração dos dados do Enem 2019**